## 1. Data Loading & Processing

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df['MedHouseVal'] = california.target

# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

# Handle missing values (if any)
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Check for data inconsistencies
print("\nDescriptive statistics:")
print(df_imputed.describe())

Missing values in each column:
MedInc         0
HouseAge       0
AveRooms       0
AveBedrms      0
Population     0
AveOccup       0
Latitude       0
Longitude      0
MedHouseVal    0
dtype: int64

Descriptive statistics:
             MedInc      HouseAge      AveRooms     AveBedrms    Population  \
count  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000   
mean       3.870671     28.639486      5.429000      1.096675   1425.476744   
std        1.899822     12.585558      2.474173      0.473911   1132.462122   
min        0.499900      1.000000      0.846154      0.333333      3.000000   
25%        2.563400     18.000000      4.440716      1.006079    787.000000   
50%        3.534800     29.000000      5.229129      1.048780   1166.000000   
75%        4.743250     37.000000      6.052381      1.099526   1725.000000   
max       15.000100     52.000000    141.909091     34.066667  35682.000000   

           AveOccup      Latitude     Longitude   MedHouseVal  
co

## 2. Feature Engineering

We check if the maximum VIF value exceeds the threshold of 5. If high multicollinearity is detected, we need to address it by removing highly correlated features or using regularization techniques like Ridge or Lasso regression.

### a. Check for Multicollinearity

In [12]:
# Check for multicollinearity using VIF
features = df_imputed.drop('MedHouseVal', axis=1)
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(features.values, i) for i in range(features.shape[1])]
vif["Features"] = features.columns
print("Variance Inflation Factor (VIF):")
print(vif)

# If VIF values are greater than 5 or 10, it indicates high multicollinearity
if vif['VIF'].max() > 5:
    print("High multicollinearity detected!")
    # TODO: Address multicollinearity (e.g., remove highly correlated features, use regularization)
else:
    print("No significant multicollinearity found.")

Variance Inflation Factor (VIF):
          VIF    Features
0   11.511140      MedInc
1    7.195917    HouseAge
2   45.993601    AveRooms
3   43.590314   AveBedrms
4    2.935745  Population
5    1.095243    AveOccup
6  559.874071    Latitude
7  633.711654   Longitude
High multicollinearity detected!


### b. Feature Selection

#### i. By looking at Correlation Analysis

In [17]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df)
df_scaled = pd.DataFrame(scaled_features, columns=df.columns)

# Select important features using correlation matrix
corr_matrix = df_scaled.corr()
corr_threshold = 0.5
relevant_features_corr = corr_matrix[corr_matrix['MedHouseVal'].abs() > corr_threshold].index
df_selected_corr = df_scaled[relevant_features_corr]

In [18]:
relevant_features_corr

Index(['MedInc', 'MedHouseVal'], dtype='object')

#### ii. With statistical tests using `SelectKBest`

In [19]:
k = 5  # Number of top features to select
selector = SelectKBest(score_func=f_regression, k=k)
selector.fit(df_scaled, y)
selected_features_selectkbest = df_scaled.columns[selector.get_support()]
df_selected_selectkbest = df_scaled[selected_features_selectkbest]

In [21]:
selected_features_selectkbest

Index(['MedInc', 'HouseAge', 'AveRooms', 'Latitude', 'MedHouseVal'], dtype='object')

#### iii. With Recursive Feature Elimination (RFE)

In [22]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

n_features = 5  # Number of features to select
rfe = RFE(estimator=LinearRegression(), n_features_to_select=n_features)
rfe.fit(df_scaled, y)
selected_features_rfe = df_scaled.columns[rfe.support_]
df_selected_rfe = df_scaled[selected_features_rfe]

In [23]:
selected_features_rfe

Index(['AveRooms', 'AveBedrms', 'Latitude', 'Longitude', 'MedHouseVal'], dtype='object')

### c. Create new relevant features

In [45]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df)
poly_feature_names = poly.get_feature_names_out(df.columns)

# Select specific polynomial features
selected_poly_features = [
    'MedInc',
    'AveRooms',
    'Latitude',
    'MedHouseVal',
    'MedInc^2',
    'MedInc AveRooms',
    'AveRooms^2',
    'AveRooms MedHouseVal',
    'Latitude^2',
    'MedHouseVal^2'
]

In [46]:
selected_poly_features

['MedInc',
 'AveRooms',
 'Latitude',
 'MedHouseVal',
 'MedInc^2',
 'MedInc AveRooms',
 'AveRooms^2',
 'AveRooms MedHouseVal',
 'Latitude^2',
 'MedHouseVal^2']

In [47]:
df_poly_selected = pd.DataFrame(poly_features, columns=poly_feature_names)[selected_poly_features]

In [48]:
final_df = df_poly_selected

In [51]:
final_df

,MedInc,AveRooms,Latitude,MedHouseVal,MedInc^2,MedInc AveRooms,AveRooms^2,AveRooms MedHouseVal,Latitude^2,MedHouseVal^2
0,8.3252,6.984127,37.88,4.526,69.308955,58.144254,48.778030,31.610159,1434.8944,20.484676
1,8.3014,6.238137,37.86,3.585,68.913242,51.785271,38.914354,22.363721,1433.3796,12.852225
2,7.2574,8.288136,37.85,3.521,52.669855,60.150315,68.693192,29.182525,1432.6225,12.397441
3,5.6431,5.817352,37.85,3.413,31.844578,32.827897,33.841580,19.854621,1432.6225,11.648569
4,3.8462,6.281853,37.85,3.422,14.793254,24.161264,39.461681,21.496502,1432.6225,11.710084
...,...,...,...,...,...,...,...,...,...,...
20635,1.5603,5.045455,39.48,0.781,2.434536,7.872423,25.456612,3.940500,1558.6704,0.609961
20636,2.5568,6.114035,39.49,0.771,6.537226,15.632365,37.381425,4.713921,1559.4601,0.594441
20637,1.7000,5.205543,39.43,0.923,2.890000,8.849423,27.097675,4.804716,1554.7249,0.851929
20638,1.8672,5.329513,39.43,0.847,3.486436,9.951266,28.403708,4.514097,1554.7249,0.717409


## 3. Model Development

#### i. Select the best traditional model

In [52]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into features (X) and target (y)
X = final_df
y = final_df['MedHouseVal']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the models and their hyperparameters
models = [
    ('Linear Regression', LinearRegression()),
    ('Ridge', Ridge()),
    ('Lasso', Lasso()),
    ('Elastic Net', ElasticNet()),
    ('Decision Tree', DecisionTreeRegressor()),
    ('Random Forest', RandomForestRegressor()),
    ('Gradient Boosting', GradientBoostingRegressor())
]

# Define the hyperparameter grids for each model
param_grids = [
    {},  # Linear Regression has no hyperparameters
    {'alpha': [0.1, 1.0, 10.0]},  # Ridge
    {'alpha': [0.1, 1.0, 10.0]},  # Lasso
    {'alpha': [0.1, 1.0, 10.0], 'l1_ratio': [0.2, 0.5, 0.8]},  # Elastic Net
    {'max_depth': [3, 5, 7], 'min_samples_split': [2, 5, 10]},  # Decision Tree
    {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]},  # Random Forest
    {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.1, 1.0], 'max_depth': [3, 5, 7]}  # Gradient Boosting
]

# Perform grid search for each model
best_models = []
for name, model in models:
    grid_search = GridSearchCV(estimator=model, param_grid=param_grids[models.index((name, model))], cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_models.append((name, grid_search.best_estimator_))
    print(f"Best hyperparameters for {name}: {grid_search.best_params_}")

# Evaluate the models on the test set
for name, model in best_models:
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name}:")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R-squared: {r2:.4f}\n")

Best hyperparameters for Linear Regression: {}
Best hyperparameters for Ridge: {'alpha': 0.1}
Best hyperparameters for Lasso: {'alpha': 0.1}
Best hyperparameters for Elastic Net: {'alpha': 0.1, 'l1_ratio': 0.2}
Best hyperparameters for Decision Tree: {'max_depth': 7, 'min_samples_split': 10}
Best hyperparameters for Random Forest: {'max_depth': 7, 'n_estimators': 300}
Best hyperparameters for Gradient Boosting: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}
Linear Regression:
Mean Squared Error: 0.0000
R-squared: 1.0000

Ridge:
Mean Squared Error: 0.0000
R-squared: 1.0000

Lasso:
Mean Squared Error: 0.1272
R-squared: 0.9029

Elastic Net:
Mean Squared Error: 0.0584
R-squared: 0.9555

Decision Tree:
Mean Squared Error: 0.0001
R-squared: 0.9999

Random Forest:
Mean Squared Error: 0.0000
R-squared: 1.0000

Gradient Boosting:
Mean Squared Error: 0.0000
R-squared: 1.0000



#### ii. Develop a Bayesian model using Scikit-Learn's Bayesian Ridge model

In [55]:
from sklearn.linear_model import BayesianRidge

# Create and train the BayesianRidge model
bayesian_ridge = BayesianRidge()
bayesian_ridge.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_bayesian_ridge = bayesian_ridge.predict(X_test)
mse_bayesian_ridge = mean_squared_error(y_test, y_pred_bayesian_ridge)
r2_bayesian_ridge = r2_score(y_test, y_pred_bayesian_ridge)

print("BayesianRidge:")
print(f"Mean Squared Error: {mse_bayesian_ridge:.4f}")
print(f"R-squared: {r2_bayesian_ridge:.4f}\n")

BayesianRidge:
Mean Squared Error: 0.0000
R-squared: 1.0000



#### iii. Develop my own Bayesian Regression Model

In [56]:
import numpy as np
from scipy.stats import multivariate_normal

class BayesianRegression:
    def __init__(self, alpha=1.0, beta=1.0):
        self.alpha = alpha
        self.beta = beta
        self.mean = None
        self.cov = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        X_train = np.concatenate((np.ones((n_samples, 1)), X), axis=1)

        precision = self.alpha * np.eye(n_features + 1) + self.beta * X_train.T.dot(X_train)
        self.cov = np.linalg.inv(precision)
        self.mean = self.beta * self.cov.dot(X_train.T).dot(y)

    def predict(self, X):
        n_samples = X.shape[0]
        X_test = np.concatenate((np.ones((n_samples, 1)), X), axis=1)
        y_pred = X_test.dot(self.mean)
        return y_pred

# Create and train the Bayesian Regression model
bayesian_regression = BayesianRegression(alpha=1.0, beta=1.0)
bayesian_regression.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred_bayesian_regression = bayesian_regression.predict(X_test)
mse_bayesian_regression = mean_squared_error(y_test, y_pred_bayesian_regression)
r2_bayesian_regression = r2_score(y_test, y_pred_bayesian_regression)

print("Bayesian Regression (Mathematical Implementation):")
print(f"Mean Squared Error: {mse_bayesian_regression:.4f}")
print(f"R-squared: {r2_bayesian_regression:.4f}")

Bayesian Regression (Mathematical Implementation):
Mean Squared Error: 0.0000
R-squared: 1.0000


## 4. Model Training & Evaluation